In [3]:
!nvidia-smi

Tue Jun 13 19:11:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:09:00.0 Off |                  N/A |
|  0%   25C    P8    22W / 350W |      5MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:41:00.0 Off |                  N/A |
|  0%   

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [15]:
from train_utils import TrainWrap
import pandas as pd


model_ckpt = 'roberta-base'

tw = TrainWrap(model_ckpt)

#train_text = pd.read_csv('data/train.tsv', delimiter='\t')
#val_text = pd.read_csv('data/validate.tsv', delimiter='\t')

#train, train_enc = tw.preprocess(train_text, 'implicit_class')

train_text = pd.read_csv("data/SBIC.v2/SBIC.v2.agg.trn.csv")
val_text = pd.read_csv("data/SBIC.v2/SBIC.v2.agg.dev.csv")
#test_text = pd.read_csv('data/test.tsv', delimiter='\t')

In [17]:
from collections import Counter
import ast

train_text['targetCategory'] = [ast.literal_eval(x) for x in train_text['targetCategory']]
val_text['targetCategory'] = [ast.literal_eval(x) for x in val_text['targetCategory']]
counts = Counter([y for x in train_text['targetCategory'].to_list() for y in x])
labels = list(counts.keys())


ValueError: malformed node or string: []

In [58]:
from ast import literal_eval
import html

import re


def process_alpaca(text, label, instruction=None):
#     instruction = defs
    
    out = {'instruction': [],
           'input': [],
           'output': []
          
          }
    
    for i in range(len(text)):
        outp = literal_eval(label[i])
        if len(outp) == 0:
            outp = 'N/A'
        else:
            outp = ', '.join(outp).strip()
            
        inp = text[i].strip()
        inp = re.sub(r'(u/.*? |@.*?( |$))', "<user> ", inp)
        inp = re.sub('\u2019', "'", inp)
        inp = re.sub('[“”]', '"', inp)
        inp = re.sub(r'\\u[a-f0-9]{4}', "'", inp)
        inp = re.sub(r'\(https://.*\)', "(URL)", inp)

        inp = re.sub(r'[\s]{2,}', " ", inp)
        inp = html.unescape(inp).encode('ascii', errors='ignore').decode()
        
        out['instruction'].append(instruction)
        out['input'].append(inp)
        out['output'].append(outp)
    return out

In [61]:
instruction = "Please reply with the group or attribute targeted by this statement. If there is no target, output N/A."

train_enc = process_alpaca(train_text['post'].to_list(), train_text['targetMinority'].to_list(), instruction=instruction)

In [68]:
len(train_enc['input']), len(train_text)

(35424, 35424)

In [63]:
from datasets import Dataset

train = Dataset.from_dict(train_enc)

In [65]:
train.to_json('targetCategory.json')

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

9308127